In [1]:
""" util """
import sys
import os
from tqdm import tqdm
import time

""" Hardware """
import board
import busio
import adafruit_lsm9ds1


""" analyzing """
import pandas as pd
import numpy as np

""" display"""
import matplotlib.pyplot as plt
# pip3 install adafruit-circuitpython-lsm9ds1
#https://www.st.com/content/ccc/resource/technical/document/datasheet/1e/3f/2a/d6/25/eb/48/46/DM00103319.pdf/files/DM00103319.pdf/jcr:content/translations/en.DM00103319.pdf
# I2C connection:

""" Ctrl """
sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))
from  madgwick import madgwickahrs #for madgwick filter

In [4]:
class LSM9DS():
    def __init__(self):
        self.i2c = busio.I2C(board.SCL, board.SDA)
        self.sensor = adafruit_lsm9ds1.LSM9DS1_I2C(self.i2c)

    """ Read values """
    # acceleration
    def readAccelValue(self):
        x, y, z = self.sensor.acceleration
        return x, y, z 

    # magnetometer
    def readMagValue(self):
        x, y, z = self.sensor.magnetic
        return x, y, z 

    # gyroscope
    def readGyroValue(self):
        x, y, z = self.sensor.gyro
        return x, y, z 

    # temperature
    def readTempValue(self):
        temp = self.sensor.temperature
        return temp

    """ Print """
    def printAccelValue(self, obs):
        x, y, z  = obs
        print("Acceleration (m/s^2): ({0:0.3f},{1:0.3f},{2:0.3f})".format(x, y, z))

    def printMagValue(self, obs):
        x, y, z  = obs
        print("Magnetometer (gauss): ({0:0.3f},{1:0.3f},{2:0.3f})".format(x, y, z))

    def printGyroValue(self, obs):
        x, y, z  = obs
        print("Gyroscope (degrees/sec): ({0:0.3f},{1:0.3f},{2:0.3f})".format(x, y, z))

    def printTempValue(self,temp):
        print("Temperature: {0:0.3f}C".format(temp))

In [35]:
""" Get 9-axis data"""

sensor = LSM9DS()
ls_sensor = ['accl', 'mag', 'gyro']
ls_feature = ls_sensor+['cumsum_gyro', 'angle']
records = { _key : np.array([]) for _key in ls_sensor } 

madgwick_obj = madgwickahrs.MadgwickAHRS()
records['angle'] = []

for i in tqdm(range(200)):
    time.sleep(0.1)
    records['accl'] = np.append(records['accl'], np.array(sensor.readAccelValue()))
    records['mag'] = np.append(records['mag'], np.array(sensor.readMagValue()))
    records['gyro'] = np.append(records['gyro'], np.array(sensor.readGyroValue()))
    #  madgwick_obj
    madgwick_obj.update( 
        np.array(sensor.readGyroValue()),
        np.array(sensor.readAccelValue()),
        np.array(sensor.readMagValue()))

    records['angle'] =  np.append(records['angle'],madgwick_obj.quaternion.to_euler_angles())

# reshape for making df
for _key in ls_sensor+['angle']:
    records[_key] = records[_key].reshape(-1,3)
records['cumsum_gyro'] = (records['gyro']*0.1).cumsum(axis=0)

# to data frame
dfs = {}
for _key in ls_feature:
    dfs[_key] = pd.DataFrame(records[_key])
    dfs[_key].columns = ['x','y', 'z']
dfs["angle"].columns = ['roll', 'pitch', 'yaw']

100%|██████████| 200/200 [00:24<00:00,  8.33it/s]


In [ ]:
# plot 9-axis
fig = plt.figure(figsize=(15, 10), facecolor='white', 
                 linewidth=10, edgecolor='white',tight_layout=True)
for _i, _key in enumerate(ls_feature):
    ax = fig.add_subplot(len(ls_feature),1,_i+1)
    if _key == 'cumsum_gyro':
        dfs[_key].loc[:,['x','y']].plot.line(ax= ax)
    else:
        dfs[_key].plot.line(ax= ax)
    ax.set_title(_key)
    ax.grid(axis='x',linestyle='--' )
    ax.grid(axis='y',linestyle='--' )
    ax.set_xlabel('Elapsed Time[100ms]')
    ax.set_ylabel('Value')
    ax.legend(loc='right')
